#### Name: 

# Deep learning from scratch
## Homework 1
### Exercise 2

-----

### General instructions

Complete the exercise listed below in this Jupyter notebook - leaving all of your code in Python cells in the notebook itself.  Feel free to add any necessary cells. 

### When submitting this homework:

Make sure you have put your name at the top of each file
    
Make sure all output is present in your notebook prior to submission

----

## <span style="color:#a50e3e;">Exercise 2. </span> On the inefficiency of the forward mode of AD when the input dimension is large (10 points)

In [Section 3.9 of the lecture notes](https://jermwatt.github.io/mlrefined/blog_posts/3_Automatic_differentiation/3_9_AD_reverse_mode.html), in order to motivate the reverse mode of automatic differentiation (AD), we show the forward mode applied to a simple quadratic with $N=2$ input variables, i.e., $g(w_1,w_2)=w_1^2+w_2^2$, whose computational graph is re-created below for convenience. 

<p><img src="pics/forward2.png" width="70%" height="auto"></p>

As you can see here when $N=2$ the computational graph has $5$ nodes. With the forward mode of AD we compute the full gradient at each node, resulting in $5\times 2=10$ partial derivative computations in total, of which $4$ are always zero. To be specific, these are: $\frac{\partial}{\partial w_2}w_1$, $\frac{\partial}{\partial w_2}a$, $\frac{\partial}{\partial w_1}w_2$, and $\frac{\partial}{\partial w_1}b$. This means that $\frac{4}{10}=40\%$ of our partial derivatives are zero. 

Additionally, we show the computational graph corresponding to another simple quadratic with $N=4$ input variables, i.e., $g(w_1,w_2,w_3,w_4)=w_1^2+w_2^2+w_3^2+w_4^2$

<p><img src="pics/forward4.png" width="80%" height="auto"></p>

Here, the computational graph consists of $11$ nodes, resulting in $11\times 4=44$ partial derivative computations in total, of which $24$ are always zero, meaning that $\frac{24}{44}\approx 54\%$ of our partial derivative calculations are redundant.      

In this Exercise you will calculate the sparsity ratio for a simple quadratic with $N$ input variables, i.e., $g(w_1,\ldots,w_N)=w_1^2+\ldots+w_N^2$.

$$\text{sparsity ratio}= \frac{\text{number of partial derivatives that are always zero}}{\text{total number of partial derivatives}}$$

### <strong>Question 1:</strong> Derive a formula for sparsity ratio in terms of $N$. You must show your work to receive full credit. 

Hint 1: For simplicity, you can first assume $N=2^m$ and find the answer in terms of $m$. Then, simply replace $m$ with $\text{log}_2 N$.

Hint 2: Assuming $N$ will be large you may employ reasonable approximations to simplify your final answer.

### Answer:

For an input size $N=2^m$, the first layer of the computational graph would have $N$ nodes. Each node would have $N-1$ zeros in its gradient matrix. Therfore, the number of zeros for the first layer of the coputation graph is:

$$N(N-1)$$

The next layer of the graph will also have $N(N-1)$ zeros as there is no interaction between the nodes moving from the input to the $(.)^2$ operation. The layer after that would have $\frac{N}{2}$ nodes and $\frac{N}{2}(N-1)$ zeros. The next layer would have $\frac{N}{4}$ nodes and $\frac{N}{4}(N-4)$ zeros, and the layer after that would have $\frac{N}{8}$ nodes and $\frac{N}{8}(N-8)$ zeros. The number of zeros per layer can thus be generalized to:

$$\frac{N}{2^i}(N-2^i)$$

where $i=0,1...,m-1$. To get the number of zeros for the whole tree, we can use the following equation:

$$T_{zeros}=N(N-1)+\sum_{i=0}^{log_2(N)-1} \frac{N}{2^i}(N-2^i)$$

The total number of partial derivatives can be found using the following equation:
$$T_{partials}=N^2+\sum_{i=0}^{log_2(N)} N(\frac{N}{2^i})$$

Finally, the sparsity ratio can be found by dividing $T_{zeros}$ by $T_{partials}$.

### <strong>Question 2:</strong> Using your formula find the sparsity ratio when $N=100$. What does this ratio converge to as $N$ grows large?   

### Answer:

Since $log_2(100)$ is not an integer, 128 was used instead ($log_2(128)=7$).

In [13]:
import math

N = 128
m = int(math.log(N)/math.log(2))

def Tzeros(N,m):
    z = N*(N-1)
    for i in range(m):
        z_layer = (N/2**i)*(N-2**i)
        z += z_layer
    return z

def Tpartials(N,m):
    p = N**2
    for i in range(m+1):
        p_layer = N*(N/2**i)
        p += p_layer
    return p
        
Tz = Tzeros(N,m)
Tp = Tpartials(N,m)
sparsity = Tz/Tp

print("N:",N)
print("m:",m)
print("Total # of zero partials:",Tz)
print("Total # of partials:",Tp)
print("Sparsity Ratio:",sparsity)
    

N: 128
m: 7
Total # of zero partials: 47872.0
Total # of partials: 49024.0
Sparsity Ratio: 0.9765013054830287


As N grows larger and larger, the sparsity ratio converges to 1. This makes sense because the number of partials that are always zero exponentially grows with the number of inputs.